<a href="https://colab.research.google.com/github/JoeHasell/test/blob/main/Fiona's_maternal_mortality_notebook_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
R.version.string

print(installed.packages())

Some new text

In [ ]:
install.packages("janitor")

library(readxl)
library(dplyr)
library(janitor)
library(tidyr)
library(ggplot2)
date <- Sys.Date()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




Download data from Gapminder and save it locally

In [ ]:
dir.create("data/input", recursive = TRUE)
download.file('https://www.gapminder.org/documentation/documentation/gapdata010.xls', paste0('data/input/',date,'_gapminder_data.xls'))

Read in the Gapminder data

In [ ]:

df <- read_xls(paste0('data/input/',date,'_gapminder_data.xls'), col_names = TRUE)
df <- df[-(1:16),]
head(df)

Warning message:
“Expecting logical in M1150 / R1150C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1151 / R1151C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1152 / R1152C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1153 / R1153C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1154 / R1154C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1155 / R1155C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1156 / R1156C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1157 / R1157C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1158 / R1158C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1159 / R1159C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1160 / R1160C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M1161 / R1161C13: got 'repro.age 15-45'”
Warning message:
“Expecting logical in M

Country,year,MMR,Live Births,Maternal deaths,Women reproductive age (15-49),MM-rate,...8,Skilled attendance %,"Source MMR, maternal death and Live birth",Comment,Source women reproductive age,Comments
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<chr>,<chr>,<chr>,<lgl>
Australia,1871-1980,643,NA,NA,NA,NA,NA,NA,"Loudon, Death in Childbirth,1992",only State of Virginia,NA,NA
Australia,1881-1990,592,NA,NA,NA,NA,NA,NA,"Loudon, Death in Childbirth, 1992",only State of Virginia,NA,NA
Australia,1891-1900,560,NA,NA,NA,NA,NA,NA,"Loudon, Death in Childbirth, 1992",only State of Virginia,NA,NA
Australia,1901-5,609,NA,NA,940332,0,NA,NA,"Loudon, Death in Childbirth, 1992",only State of Virginia,"Mitchell, B. International Historical Statistics. Europe 1750-2000. Fifth Edition, Palgrave, Maximillan, 2003",NA
Australia,1906,538,107890,NA,NA,NA,NA,NA,"Loudon, Death in Childbirth, 1992",only State of Virginia,NA,NA
Australia,1907,556.42654535238842,110347,614,NA,NA,NA,NA,MD: Australian Institute of Health and Welfare (AIHW) LB and women reproductive age http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/3105.0.65.0012006?OpenDocument,NA,NA,NA


The first three year ranges seem to be wrong as in another source (https://docs.google.com/spreadsheets/u/0/d/14ZtQy9kd0pMRKWg_zKsTg3qKHoGtflj-Ekal9gIPZ4A/pub?gid=1#) they cover only 10 years rather than 110. I will replace them manually here with the middle of each decade, as is used in the other data from the same source.

There are also two cases where the year for Finland has been entered wrongly three times, there are two 1772s, 1775s & 1967s, the second of each should be 1872, 1875 and 1967 respectively.

There are also two errors for New Zealand, and one error for both Swecen and the US.

In [ ]:
df$year[1:3] <- c(1875, 1885,1895)
df['year'][df$year == '1772' & df$Country == 'Finland' & df$`Maternal deaths`== 487,] <- '1872'
df['year'][df$year == '1775' & df$Country == 'Finland' & df$`Maternal deaths` == 629,] <- '1875'
df['year'][df$year == '1967' & df$Country == 'Finland' & df$`Maternal deaths` == 77,] <- '1957'
df['year'][df$year == '1967' & df$Country == 'Sweden' & df$`Maternal deaths` == 39,] <- '1957'
df['year'][df$year == '1967' & df$Country == 'United States' & df$`Maternal deaths` == 1766.28,] <- '1957'
df['year'][df$year =='1989-02'& df$Country == 'New Zealand',] <- '1889-02'
#Dropping an erroneous row for NZ in 1950 - there are two values but the first is incorrect. You can using the maternal deaths and live births columns. 
df <- df[-which(df$year == '1950' & df$Country == 'New Zealand' & df$MMR == 90),]

For some rows a range of years is given for a particular maternal mortality rate. We use this function to find the mid-value of years given.

In [ ]:
mid_year <- function(year) {
  if(grepl("-", year)){
    year_split <-  unlist(strsplit(year, split = "-"))
    year_begin <- year_split[1]
    year_end_len <- nchar(year_split[2])
    year_end_pref <- 4 - year_end_len
    
    if (year_end_pref > 0) {
      year_pref <- substr(year_begin, 1, year_end_pref)
      year_end <- as.numeric(paste(c(year_pref, year_split[2]),collapse = ""))
      year_out <- round(mean(c(as.numeric(year_begin), year_end)))
    }else{
      year_out <- round(mean(as.numeric(c(year_begin,year_split[2]))))
    }
  }else{
    year_out <- year
  }
  
  return(year_out)
}
year_out_all <- unlist(lapply(df$year, mid_year))
df$mid_year <- as.numeric(year_out_all)

Let's check the mid-year estimates have worked as expected:

In [ ]:
df %>% select(year, mid_year) %>% filter(grepl("-",year)) %>% distinct()

year,mid_year
<chr>,<dbl>
1901-5,1903
1856-60,1858
1871-80,1876
1881-90,1886
1891-1900,1896
1876-1880,1878
1881-1885,1883
1886-1890,1888
1891-1895,1893


Keeping only the columns we want to take forward.

In [ ]:
df <- df %>% 
  select(entity = Country, year = mid_year, maternal_mortality_rate = MMR)